In [6]:
from piec.drivers.utilities import PiecManager
from piec.drivers.keysight81150a import Keysight81150a
from piec.drivers.keysightdsox3024a import Dsox3024a
pm = PiecManager()

In [7]:
pm.parse_resources()

('GPIB0::7::INSTR', 'GPIB0::8::INSTR')

In [8]:
awg = Keysight81150a("GPIB0::8::INSTR")
scope = Dsox3024a("GPIB0::7::INSTR")

In [9]:
awg.idn()

'Agilent Technologies,81150A,MY53821602,3.0.0.0-4.6\n'

In [20]:
scope.idn()

'AGILENT TECHNOLOGIES,DSO-X 3024A,MY63080078,02.65.2021030741\n'

In [11]:
#Setup functions
#Set up the pulse params
awg.configure_wf(channel='1', func="PULS", voltage="1.0", offset="0.0", frequency="1.0e7", duty_cycle="50")
#Sets the trigger source to manual
awg.configure_trigger(trigger_source="MAN")
#turns the output on
awg.output_enable(channel='1')

In [13]:
#Configure the scope
def configure_oscilloscope(instrument, channel:str = 1):
        """
        Configures the Oscilloscope to capture the waveform.
        """
        instrument.initialize()
        instrument.configure_timebase(time_base_type='MAIN', reference='CENTer', time_scale=f'{100e-9}', position=f'{0}') #this should be made general
        instrument.configure_channel(channel=f'{channel}', voltage_scale=f'{1}', impedance='FIFT')#set both to 50ohm
        instrument.configure_trigger_characteristics(trigger_source='CHAN1', low_voltage_level='0.75', high_voltage_level='0.95', sweep='NORM')
        instrument.configure_trigger_edge(trigger_source='CHAN1', input_coupling='DC')
configure_oscilloscope(scope)

In [15]:
awg.send_software_trigger()
#bet = scope.query_wf()

In [23]:
scope.setup_wf()
bet = scope.query_wf()

In [24]:
bet

({'format': np.int16(0),
  'type': np.int16(0),
  'points': np.int32(1000),
  'count': np.int32(1),
  'x_increment': np.float64(1e-09),
  'x_origin': np.float64(-5e-07),
  'x_reference': np.int32(0),
  'y_increment': np.float32(0.040201),
  'y_origin': np.float32(0.0),
  'y_reference': np.int32(0)},
 [np.float64(-5e-07),
  np.float64(-4.99e-07),
  np.float64(-4.979999999999999e-07),
  np.float64(-4.97e-07),
  np.float64(-4.96e-07),
  np.float64(-4.95e-07),
  np.float64(-4.94e-07),
  np.float64(-4.93e-07),
  np.float64(-4.92e-07),
  np.float64(-4.909999999999999e-07),
  np.float64(-4.9e-07),
  np.float64(-4.89e-07),
  np.float64(-4.88e-07),
  np.float64(-4.87e-07),
  np.float64(-4.86e-07),
  np.float64(-4.85e-07),
  np.float64(-4.839999999999999e-07),
  np.float64(-4.83e-07),
  np.float64(-4.82e-07),
  np.float64(-4.809999999999999e-07),
  np.float64(-4.8e-07),
  np.float64(-4.79e-07),
  np.float64(-4.78e-07),
  np.float64(-4.769999999999999e-07),
  np.float64(-4.76e-07),
  np.float64(-

In [15]:
bet = scope.query_wf()

In [16]:
bet

({'format': np.int16(0),
  'type': np.int16(0),
  'points': np.int32(62500),
  'count': np.int32(1),
  'x_increment': np.float64(1.6e-08),
  'x_origin': np.float64(-0.0005),
  'x_reference': np.int32(0),
  'y_increment': np.float32(0.201005),
  'y_origin': np.float32(0.0),
  'y_reference': np.int32(128)},
 [np.float64(-0.0005),
  np.float64(-0.000499984),
  np.float64(-0.000499968),
  np.float64(-0.000499952),
  np.float64(-0.000499936),
  np.float64(-0.00049992),
  np.float64(-0.000499904),
  np.float64(-0.000499888),
  np.float64(-0.0004998720000000001),
  np.float64(-0.000499856),
  np.float64(-0.00049984),
  np.float64(-0.0004998240000000001),
  np.float64(-0.000499808),
  np.float64(-0.000499792),
  np.float64(-0.0004997760000000001),
  np.float64(-0.00049976),
  np.float64(-0.000499744),
  np.float64(-0.0004997280000000001),
  np.float64(-0.000499712),
  np.float64(-0.000499696),
  np.float64(-0.00049968),
  np.float64(-0.000499664),
  np.float64(-0.000499648),
  np.float64(-0.00

In [14]:
#send the trigger
awg.send_software_trigger()